# Lab 4: Competitive learning on the MNIST database

## Learning Outcomes
- Understand how competive learning works.
- Ability to develop and tune a simple neural network using competitive learning.

## Lab Overview

### Competitive learning
- The aim of this lab is to implement the standard competitive learning algorithm on a one layer network and use it to cluster the hand written digits (MNIST) data set.
- Clustering: given a set of datapoints we want to group them based on their similarity. Each one of these groups is called cluster.
- Using competitive learning, the output neurons compete amongst themselves to be activated (fire).
- In contrast with the Hebbian learning, here only one (output) neuron (or group of neurons) can be active at a time.
- The output neuron that wins the competition is is often called the winner-take-all neuron.


<img src="competitive.png" width="400">


### Main steps for competitive learning implementation:
1. Generate a set of output neurons with random weights.
2. Choose a random input pattern and calculate the activation (total input) of each of the output neurons using:
$$ h_i = \sum_j W_{ij} x^{\mu}_j $$
3. Detect the winner neuron and update only its weights (the weights of the other neurons are not updated):
$$ \Delta W_{kj} = \eta ( x^{\mu}_j - W_{kj}) $$
    where $k$ is the index of winning neuron, $\eta$ is the learning rate and $x^{\mu}_j$ is the input vector.
4. Repeat from step 2 until the weights are no longer changing, or change less than a set threshold, or a set maximum number of iterations has been reached.

### Reconstruction Error and Exponential Moving Average
A measure of how well the network is clustering the datapoints can be given using the *Reconstruction Error*. For an individual datapoint the reconstruction error is the squared difference between the datapoint and the weight vector of the cluster it belongs to. Remember, in competitive learning the data point belongs to the cluster for which the neuron activation is the largest. If $k$ is the winning neuron then this individual error for pattern $\mu$ is
$$ E^{\mu} = \sum_j ( x^{\mu}_j - W_{kj})^2 $$ 
or in vector notation we can write this as (assuming both are column vectors): 
$$ E^{\mu} = (\mathbf{x^{\mu}} - \mathbf{W}_k)^T (\mathbf{x^{\mu}} - \mathbf{W}_k) $$.

The total reconstruction error can be calculated by summing over the whole dataset (as in the lectures):
$$ E = \sum_\mu E^{\mu} = \sum_\mu \sum_j ( x^{\mu}_j - W_{kj})^2, $$
or alternatively we will track an exponential moving average of the individual error. This applys a smoothing to individual errors and allows us to monitor how the error is dropping. The average is calculated using
$$ \bar{E}_t = (1-\alpha) \bar{E}_{t-1}  + \alpha E^{\mu},$$
where $\bar{E}_t$ is the moving average at iteration $t$ and $\alpha$ is the smoothing rate. For the first iteration $t=0$, we need to consider the boundary condition, since we have no value $\bar{E}_{t-1}$ we have a special case and can start the moving average using
$$ \bar{E}_{0} = E^{\mu}, $$
i.e we set the moving average to first measurement of the error.

### Correlation Matrix of the Prototypes

As your network finds clusters and the prototypes (weights) come to represent these clusters, you will notice that there are similarities between different prototypes. We can analyse these similarities by calculating the correlation between two prototypes, similar to how in class we looked at the correlation between inputs. In case of the prototypes we are looking for similarities over the input dimension (i.e 2nd dimension of the weight matrix, so the elements of the correlation matrix are
$$ C_{ij} = \sum_k W_{ik} W_{jk}, $$
where $C_{ij}$ is the correlation between prototypes (weights) $i$ and $j$. This will be symmetric (consider swapping $i$ and $j$ in the above equation) and the diagonal will represent the squared magnitude ($|\mathbf{W}_i|^2$) of the prototypes. If the weights are no longer normalised you should consider normalising them before calculating this so only similarity is measured and not the difference in magnitudes.


### Exercise
Using the skeleton code below implement the competitive learning algorithm as detailed above. 
Please choose the number of output units such that you can capture all possible clusters, and tune the network such that it will learn quickly and result in as few dead units as possible. One suggestion is to add noise to the decision neurons, in combination with an appropriate decaying learning rate, but you are free to apply other techniques. In this example you are able to visualise both data and prototypes, and can easily locate the dead units. The vectors provided are 28x28 images and you can reshape them to see the digits. 

You will need to do the following :
1. Create a figure showing the average error as a function of time. When has your network has sufficiently learned from the data? Such a curve may be more informative on semi-log or log-log axes.
2. Propose a method for detecting dead units, without using the visualisation of the prototypes.
3. Create a figure of the prototypes (weights). How many prototypes did your network find?
4. Calculate the correlation matrix of the prototypes. How can you use this information to find similarities between the prototypes?

### Some optimisation and tuning methods
You may find that your training results in a lot of dead units. To reduce these you may need to consider the following optimisation of tuning methods: 
1. Normalised input or initial weights.
2. Noise addition on the weights.
3. Decaying learning rate.
4. Leaky learning: update the weights of the losers as well as winners but with a much smaller learning rate.
5. Update the winners and neighbouring losers.

In [1]:
import numpy as np
import numpy.matlib
import math
import matplotlib

import matplotlib.pyplot as plt

from sklearn.preprocessing import normalize #normalize
from numpy import random #random noise

#### Load the data set

In [2]:
# Load the training dataset
import scipy.io as spio
mat = spio.loadmat('emnist_capletters_AtoJ.mat', squeeze_me=True)

train = mat['train_images'] # data

for data in train:                  #normalised
  data /= np.sqrt(np.dot(data, data))

[n_samples, n_pixels]  = train.shape                   

print("n_samples = ", n_samples)    # n_samples = 5,000 (images)
print("n_pixels  = ", n_pixels)     # n_pixels = 784 = 28x28 pixels

FileNotFoundError: ignored

In [ ]:
# Plot one of the mnist images
plt.imshow(train[0].reshape((28, 28), order='F'))
cbar = plt.colorbar()
cbar.set_label('Pixel Intensity')
plt.show()

print('Magnitude of image vector |x| = ', np.sqrt( np.dot(train[0], train[0])))

#### Parameters and variables 

####  Implementation, training and output

In [ ]:
avg_deadunit = 0

#fig = plt.figure()
#ax = plt.subplot(111)
n_output_list = [10,15,20]

#for n_output_num in n_output_list:       
for i in range(10):
    # Parameters
    eta      = 0.05     # learning rate 
    winit    = 1     # parameter controlling magnitude of initial weights #original 1
    tmax     = 10000  # number of iterations #original 40
    n_output = 10      # number of output neurons   #change to n_output_num to 
    alpha    = 0.01   # Moving average decay factor

    # Weight matrix (rows = output neurons, cols = input neurons)
    #W = winit * np.random.rand(n_output, n_pixels)  #Initialise weight with random value

    W = np.zeros([n_output, n_pixels])     #Initialise weight with sample

    for i in range(n_output):               #Select samples from train data
      W[i] = winit * train[np.random.default_rng().integers(n_samples)]

    for i in range(n_output):
        W[i] /= np.sqrt(np.dot(W[i], W[i]))

    deadUnit = 0

    rng = np.random.default_rng()

    # Count how many times each output neuron wins
    counter = np.zeros(n_output, dtype=int)

    update_counter = np.zeros(n_output, dtype=int)  #add bias counter

    winning_neuron = np.zeros(n_samples, dtype=int)
    moving_avg_error = np.zeros(tmax)
    for t in range(tmax):
      i = rng.integers(n_samples)
      x = train[i]

      h = np.matmul(W,x) + random.rand(n_output)*1e-07 #noise
      h = h + h*(update_counter*4e-07)   #add bias

      k = np.argmax(h)

      winning_neuron[i] = k 
      counter[k] += 1
      #count update  #add bias
      update_counter += 1
      update_counter[k] = 0


      error = np.sum(np.square(x-W[k]))
      if t == 0:
          moving_avg_error[t] = error
      else:
          moving_avg_error[t] = (1-alpha)*moving_avg_error[t-1] + alpha*error
          
      dW = eta*(x-W[k])
      
      W[k] = W[k] + dW

      W[k] /= np.sqrt(np.dot(W[k], W[k]))   #normalise again after sample

      #Leaky learning
      for i in range(W.shape[0]):
        if i != k:
          dW = (eta/1000)*(x-W[i]) 
          W[i] = W[i] + dW
          W[i] /= np.sqrt(np.dot(W[i], W[i]))   #Should be uncomment if initialise weight with train sample is enable
      
      #Update the winners and neighbouring losers
      for i in range(2,6):
        k = np.argsort(h)[-i]
        dW = (eta/10)*(6-i)/10 *(x-W[k])
        W[k] = W[k] + dW
        W[k] /= np.sqrt(np.dot(W[k], W[k]))   #normalise again after sample, should be uncomment if initialise weight with train sample is enable 
  
    deadUnit = len(np.where(counter < np.max(counter)/5)[0]) #Calculate dead unit

    avg_deadunit = avg_deadunit+deadUnit  
    #ax.semilogy(moving_avg_error, label="n_output:"+str(n_output_num))
    #ax.legend()

print("Average dead unit:" + str(avg_deadunit/10))
#plt.savefig('moving_avg_error.png', bbox_inches='tight')
#plt.show()
#plt.clf()

In [ ]:
def plotPrototypes(W):      #plot weight prototypes
  plt.figure()
  
  fig, ax = plt.subplots(5, 2, figsize=(5,15))
  ax = ax.flatten()
  for i, w in enumerate(W):
    im = ax[i].imshow(w.reshape((28, 28), order='F'))

  fig.subplots_adjust(right=0.8)
  fig.colorbar(im, fig.add_axes([0.9, 0.15, 0.06, 0.7]))

  

  plt.savefig('prototypes.png', bbox_inches='tight')
  plt.show()


plotPrototypes(W)

In [ ]:
import pandas as pd

df = pd.DataFrame(W.transpose())

corrMatrix = df.corr()

plt.matshow(corrMatrix)

cbar = plt.colorbar()

plt.savefig('corrMatrix.png')

plt.show()